In [159]:
import numpy as np

In [160]:
foundation_path = "/home/bxai1/BXAI/Project/NeuroTalk-decoder/"

In [161]:
sub_number = "sub001"

In [162]:
csp_path = foundation_path + f"CSP/{sub_number}/"

In [163]:
from scipy.io import loadmat
# Load the .mat file
data = loadmat(csp_path + 'epoch_data.mat')
mat_key = 'fv_te_imagined', 'fv_te_spoken', 'fv_tr_imagined', 'fv_tr_spoken', 'fv_val_imagined', 'fv_val_spoken'


In [164]:
print(data['fv_tr_imagined'][0][0][1].shape)

(13, 234)


In [165]:
fv_te_imagined = data[mat_key[0]]
fv_te_spoken = data[mat_key[1]]
fv_tr_imagined = data[mat_key[2]]
fv_tr_spoken = data[mat_key[3]]
fv_val_imagined = data[mat_key[4]]
fv_val_spoken = data[mat_key[5]]
all_fv = [fv_te_imagined, fv_te_spoken, fv_tr_imagined, fv_tr_spoken, fv_val_imagined, fv_val_spoken]
all_fv_name = ["fv_te_imagined", "fv_te_spoken", "fv_tr_imagined", "fv_tr_spoken", "fv_val_imagined", "fv_val_spoken"]


In [166]:
# This is a code that checks if any of the values from running vector_embedding.m have an inf value, and if there is an inf, you can check which trials have shown up. 
all_fv_ind = 5 # mat_key's length is 5.
for i in range(all_fv[all_fv_ind][0][0][0].shape[2]):
    if np.isinf(all_fv[all_fv_ind][0][0][0][:,:,i]).any():
        print(i)

In [167]:
# In all_fv_name, since train accounts for 60% of the total dataset, it is possible to know where the train file is located in the list of all_fv_name.
train_indices = []
for i in range(len(all_fv_name)):
    if all_fv_name[i].find("tr") != -1:
        train_indices.append(i)
train_indices

[2, 3]

In [168]:
import os
import numpy as np

n_trial_per_classes_in_time = 0 # 
n_trial_per_classes_in_test = 0 #


for j in range(len(all_fv)):
    # Checking folder directions
    if os.path.isdir(csp_path+all_fv_name[j]):
        pass
    else: 
        os.mkdir(csp_path+all_fv_name[j])
    if os.path.isdir(csp_path+all_fv_name[j]+'/EEG') and os.path.isdir(csp_path+all_fv_name[j]+'/label'):
        pass
    else:
        os.mkdir(csp_path+all_fv_name[j]+'/EEG')
        os.mkdir(csp_path+all_fv_name[j]+'/label')        

             

    labe = []
    # Checking how many labels in list
    for i in range(all_fv[j][0][0][1].shape[1]):
        # Convert numpy to int like using np[0] and append indicies to list
        labe.append(np.where(all_fv[j][0][0][1][:,i] == 1)[0][0])
    labe_np = np.array(labe)

    for classes in range(13):
        # Since the number of trials per class is different, you must specify the time of training, value, and test separately.
        if j in train_indices:
            n_tr_indices = all_fv[j][0][0][0][:,:,np.where(labe_np==classes)[0]].shape[2]
        elif j not in train_indices:
            n_te_indices = all_fv[j][0][0][0][:,:,np.where(labe_np==classes)[0]].shape[2]
            
        # globals()["{}_CSP_{}_{}".format(all_fv_name[j], classes, cnt)] = np.zeros((16,104))
        # Need to Checking -inf. So You have to use np.inf to remove the correspond label     
        for n in range(all_fv[j][0][0][0][:,:,np.where(labe_np==classes)[0]].shape[2]):
            globals()["{}_CSP_{}_{}".format(all_fv_name[j], classes, n)] = np.zeros((16,104))
            globals()["{}_label_{}_{}".format(all_fv_name[j], classes, n)] = np.array([0]*13)
            if not np.isinf(all_fv[j][0][0][0][:,:,np.where(labe_np==classes)[0][n]]).any():
                globals()["{}_CSP_{}_{}".format(all_fv_name[j], classes, n)] = all_fv[j][0][0][0][:,:,np.where(labe_np==classes)[0][n]]
                
                globals()["{}_label_{}_{}".format(all_fv_name[j], classes, n)][classes] = 1

            else:
                pass

In [169]:
import pandas as pd

num_class = 13  # Number of classes

for i in range(len(all_fv_name)):
    # Since the number of trials per class is different, you must specify the time of training, value, and test separately.
    if i in train_indices:
        n_indices = n_tr_indices
    elif i not in train_indices:
        n_indices = n_te_indices

    for classes in range(num_class):
        for n in range(n_indices):
            # If error processing occurs, the file is excluded because there is a nan value or an inf value as a result of CSP.
            try:
                csp_var_name = f"{all_fv_name[i]}_CSP_{classes}_{n}"
                label_var_name = f"{all_fv_name[i]}_label_{classes}_{n}"
                df_CSP= pd.DataFrame(globals()["{}_CSP_{}_{}".format(all_fv_name[i], classes, n)]).T
                df_CSP_label = pd.DataFrame(globals()["{}_label_{}_{}".format(all_fv_name[i], classes, n)]).T
                # The characteristic of the trainloader is to load files in the order of the names of the files. Therefore, when setting names, it is possible to set them in the desired order.
                if classes <= 8:
                    if n < 10:
                        df_CSP.to_csv(csp_path + all_fv_name[i]+"/"+ "EEG/" + all_fv_name[i] + "_"+ "00" + str(classes + 1) + "_00"+ f"{n}"+ ".csv", index=False, header=False)
                        df_CSP_label.to_csv(csp_path + all_fv_name[i]+"/"+ "label/" + all_fv_name[i] + "_"+ "00" + str(classes + 1) +"_00"+ f"{n}" +".csv", index=False, header=False)
                    else:
                        df_CSP.to_csv(csp_path + all_fv_name[i]+"/"+ "EEG/" + all_fv_name[i] + "_"+ "00" + str(classes + 1) + "_0"+ f"{n}"+ ".csv", index=False, header=False)
                        df_CSP_label.to_csv(csp_path + all_fv_name[i]+"/"+ "label/" + all_fv_name[i] + "_"+ "00" + str(classes + 1) +"_0"+ f"{n}" +".csv", index=False, header=False)                        
                else:
                    if n < 10:
                        df_CSP.to_csv(csp_path + all_fv_name[i]+"/"+ "EEG/" + all_fv_name[i] + "_"+ "0" + str(classes + 1) +"_00"+ f"{n}"+".csv", index=False, header=False)  
                        df_CSP_label.to_csv(csp_path + all_fv_name[i]+"/"+ "label/" + all_fv_name[i] + "_"+ "0" + str(classes + 1) +"_00"+f"{n}"+".csv", index=False, header=False) 
                    else:
                        df_CSP.to_csv(csp_path + all_fv_name[i]+"/"+ "EEG/" + all_fv_name[i] + "_"+ "0" + str(classes + 1) +"_0"+ f"{n}"+".csv", index=False, header=False)  
                        df_CSP_label.to_csv(csp_path + all_fv_name[i]+"/"+ "label/" + all_fv_name[i] + "_"+ "0" + str(classes + 1) +"_0"+f"{n}"+".csv", index=False, header=False)                         
            except Exception as e:
                print(f"Error processing {csp_var_name} or {label_var_name}: {e}")
                pass

In [170]:
folder_list = os.listdir(csp_path)[:]
folder_list.sort()
folder_list[4:]

['fv_te_imagined',
 'fv_te_spoken',
 'fv_tr_imagined',
 'fv_tr_spoken',
 'fv_val_imagined',
 'fv_val_spoken']

In [171]:
zero_EEG = []
folder_list = os.listdir(csp_path)
folder_list.sort()
folder_list = folder_list[4:]
for folder in folder_list:
    folder_EEG = sorted(os.listdir(csp_path + folder+"/EEG"))
    for i in range(len(folder_EEG)):
        if folder_EEG[i][0] == '.':
            continue
        if pd.read_csv(csp_path + folder+"/EEG/"+folder_EEG[i], header = None).values.all() == 0:
            zero_EEG.append(folder_EEG[i])
            print(folder_EEG[i])
        

fv_tr_spoken_005_006.csv


In [172]:
import shutil

fv_data = os.listdir(csp_path)
fv_data.sort()
fv_data = fv_data[4:]
fv_data

['fv_te_imagined',
 'fv_te_spoken',
 'fv_tr_imagined',
 'fv_tr_spoken',
 'fv_val_imagined',
 'fv_val_spoken']

In [174]:
import os
import shutil

def copy_with_incremented_name(full_file_name, target_dir):
    # Split the file name and extension
    base_name = os.path.basename(full_file_name)
    name, ext = os.path.splitext(base_name)

    # Set the target path for copying
    target_file = os.path.join(target_dir, base_name)
    
    # Check if the file already exists
    counter = 1
    while os.path.exists(target_file):
        # Add a number to the file name if it exists
        if counter <10:
            new_name = f"{name}(00{counter}){ext}"
        else:
            new_name = f"{name}(0{counter}){ext}"
        target_file = os.path.join(target_dir, new_name)
        counter += 1

    # Copy the file
    shutil.copy(full_file_name, target_file)

# Main code
for i in range(len(fv_data)):
    # Determine the data type (test, train, or validation)
    if fv_data[i].find("te") != -1:
        data_type = "test"
    elif fv_data[i].find("tr") != -1:
        data_type = "train"
    elif fv_data[i].find("val") != -1:
        data_type = "val"
        
    # Determine the data task (imagined or spoken)
    if fv_data[i].find("imagined") != -1:
        data_task = "im"
    elif fv_data[i].find("spoken") != -1:
        data_task = "sp"
   
    # Define paths
    path_CSP_EEG = csp_path + fv_data[i] + "/EEG/"
    path_CSP_label = csp_path + fv_data[i] + "/label/"
    path_mel = foundation_path + f"Patients/preprocessed/{sub_number}/about_voice/{sub_number}_mel/"
    path_total = [path_CSP_EEG, path_CSP_label, path_mel]
    
    for pa in path_total:
        # Source directory for copying
        source_dir = pa
        
        # Define the target directory for EEG data
        if pa.find("EEG") != -1:
            if data_task == "im":   
                target_dir = foundation_path + f"dataset/{sub_number}/" + data_type + "/ImaginedEEG_vec/"
            elif data_task == "sp":
                target_dir = foundation_path + f"dataset/{sub_number}/" + data_type + "/SpokenEEG_vec/"
                
        # Define the target directory for label data
        elif pa.find("label") != -1:
            if data_task == "im":   
                target_dir = foundation_path + f"dataset/{sub_number}/" + data_type + "/im_Y/"
            elif data_task == "sp":
                target_dir = foundation_path + f"dataset/{sub_number}/" + data_type + "/sp_Y/"
                
        # Define the target directory for mel data
        elif pa.find("mel") != -1:
            if data_task == "im":   
                target_dir = foundation_path + f"dataset/{sub_number}/" + data_type + "/im_Y_mel/"
            elif data_task == "sp":
                target_dir = foundation_path + f"dataset/{sub_number}/" + data_type + "/sp_Y_mel/"
        
        # Iterate over all files in the source directory and copy them
        for file_name in os.listdir(source_dir):
            full_file_name = os.path.join(source_dir, file_name)
            
            # Check if it is a file and copy it
            if os.path.isfile(full_file_name):
                copy_with_incremented_name(full_file_name, target_dir)
                if pa.find("mel") != -1:
                    # if 'ima_vec_num' in locals():
                    #     session_num = ima_vec_num // 13
                    # elif 'spo_vec_num' in locals():
                    #     session_num = spo_vec_num//13
                    # for i in range(session_num):
                        # copy_with_incremented_name(full_file_name, target_dir)
                # else:
                
                    
                    # Track the number of files in the imagined EEG directory
                    if pa.find("EEG") != -1 and pa.find("imagined") != -1:
                        ima_vec_num = len(os.listdir(pa))

                    # Track the number of files in the spoken EEG directory
                    elif pa.find("EEG") != -1 and pa.find("spoken") != -1:
                        spo_vec_num = len(os.listdir(pa))
                 
        # Print the number of imagined EEG files
        if pa.find("EEG") != -1 and pa.find("imagined") != -1:
            ima_vec_num = len(os.listdir(pa))
            print(f"ImaginedEEG_vec {data_type} number is: {ima_vec_num}")
        
        # Print the number of spoken EEG files
        elif pa.find("EEG") != -1 and pa.find("spoken") != -1:
            spo_vec_num = len(os.listdir(pa))
            print(f"SpokenEEG_vec {data_type} number is: {spo_vec_num}")


ImaginedEEG_vec test number is: 118
SpokenEEG_vec test number is: 117
ImaginedEEG_vec train number is: 234
SpokenEEG_vec train number is: 234
ImaginedEEG_vec val number is: 117
SpokenEEG_vec val number is: 117


In [175]:
zero_EEG_len = len(zero_EEG)

# Loop through the zero_EEG list
for i in range(zero_EEG_len):   
    # Construct the path to the CSP files
    zero_csp_path = os.path.join(csp_path, zero_EEG[i][:-12])
    zero_csv = os.path.join(zero_csp_path, zero_EEG[i])
    
    # Determine the data task: imagined or spoken
    if zero_csv.find("imagined") != -1:
        data_ta = "im"
        full_data_ta = "ImaginedEEG_vec"
    elif zero_csv.find("spoken") != -1:
        data_ta = "sp"
        full_data_ta = "SpokenEEG_vec"
        
    # Determine the data type: train, val, or test
    if zero_csv.find("tr") != -1:
        data_ty = "train"
    elif zero_csv.find("val") != -1:
        data_ty = "val"
    elif zero_csv.find("te") != -1:
        data_ty = "test"
        
    # Define paths for mel, label, and EEG data
    mel_path = foundation_path + f"dataset/{sub_number}/{data_ty}/" + data_ta + "_Y_mel/"
    label_path = foundation_path + f"dataset/{sub_number}/{data_ty}/" + data_ta + "_Y/" 
    EEG_path = foundation_path + f"dataset/{sub_number}/{data_ty}/{full_data_ta}/"
       
    # Extract label numbers from the file name
    label_num = int(zero_csv[-11:-8])
    label_num_num = int(zero_csv[-6:-4])   

    # Remove the corresponding EEG file
    os.remove(EEG_path + zero_EEG[i])
    if data_ta == "sp":
        if data_ty != "val":
            # Iterate over the files in the mel directory
            for com in os.listdir(mel_path):
                try:
                    # Check if the first label number matches
                    if int(com[17:20]) == label_num:
                        # Debug: check if label_num_num and file extension match
                        
                        # If the second label number matches, remove the file
                        if int(com[21:24]) == label_num_num:
                            os.remove(mel_path + com)
                except:
                    # If the label_num_num is 0 and the file extension is "csv", remove the file
                    if label_num_num == 0 and com[21:] == "csv":
                        os.remove(mel_path + com)

            # Iterate over the files in the label directory
            for com in os.listdir(label_path):
                try:
                    # Check if both label numbers match
                    if int(com[13:16]) == label_num:
                        if int(com[17:20]) == label_num_num:
                            os.remove(label_path + com)
                except:
                    # Ignore errors (e.g., if the file name format is unexpected)
                    pass
        else:
            # Iterate over the files in the mel directory
            for com in os.listdir(mel_path):
                try:
                    # Check if the first label number matches
                    if int(com[17:20]) == label_num:
                        # Debug: check if label_num_num and file extension match
                        
                        # If the second label number matches, remove the file
                        if int(com[21:24]) == label_num_num:
                            os.remove(mel_path + com)
                except:
                    # If the label_num_num is 0 and the file extension is "csv", remove the file
                    if label_num_num == 0 and com[21:] == "csv":
                        os.remove(mel_path + com)

            # Iterate over the files in the label directory
            for com in os.listdir(label_path):
                try:
                    # Check if both label numbers match
                    if int(com[14:17]) == label_num:
                        if int(com[18:21]) == label_num_num:
                            os.remove(label_path + com)
                except:
                    # Ignore errors (e.g., if the file name format is unexpected)
                    pass
                   
    if data_ta == "im":
        if data_ty != "val":
            # Iterate over the files in the mel directory
            for com in os.listdir(mel_path):
                try:
                    # Check if the first label number matches
                    if int(com[17:20]) == label_num:
                        # Debug: check if label_num_num and file extension match
                        
                        # If the second label number matches, remove the file
                        if int(com[21:24]) == label_num_num:
                            os.remove(mel_path + com)
                except:
                    # If the label_num_num is 0 and the file extension is "csv", remove the file
                    if label_num_num == 0 and com[21:] == "csv":
                        os.remove(mel_path + com)

            # Iterate over the files in the label directory
            for com in os.listdir(label_path):
                try:
                    # Check if both label numbers match
                    if int(com[15:18]) == label_num:
                        if int(com[19:22]) == label_num_num:
                            os.remove(label_path + com)
                except:
                    # Ignore errors (e.g., if the file name format is unexpected)
                    pass     
        else:
            # Iterate over the files in the mel directory
            for com in os.listdir(mel_path):
                try:
                    # Check if the first label number matches
                    if int(com[17:20]) == label_num:
                        # Debug: check if label_num_num and file extension match
                        
                        # If the second label number matches, remove the file
                        if int(com[21:24]) == label_num_num:
                            os.remove(mel_path + com)
                except:
                    # If the label_num_num is 0 and the file extension is "csv", remove the file
                    if label_num_num == 0 and com[21:] == "csv":
                        os.remove(mel_path + com)

            # Iterate over the files in the label directory
            for com in os.listdir(label_path):
                try:
                    # Check if both label numbers match
                    if int(com[16:19]) == label_num:
                        if int(com[20:23]) == label_num_num:
                            os.remove(label_path + com)
                except:
                    # Ignore errors (e.g., if the file name format is unexpected)
                    pass                       


In [ ]:
sorted(zero_EEG)